In [2]:
from nba_api.stats.static import players,teams
import pandas as pd
import requests
import sys
import os
import time
from datetime import datetime

def format_date_to_url(date):
    # Convert date from YYYYMMDD to datetime object
    date_obj = datetime.strptime(str(date), '%Y%m%d')
    
    # Format the date as MM%2FDD%2FYYYY
    formatted_date = date_obj.strftime('%m%%2F%d%%2F%Y')
    
    return formatted_date

# Example usage

def pull_data(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()

    if len(json["resultSets"])== 1:

        
        data = json["resultSets"][0]["rowSet"]
        #print(data)
        columns = json["resultSets"][0]["headers"]
        #print(columns)
        
        df = pd.DataFrame.from_records(data, columns=columns)
    else:

        data = json["resultSets"]["rowSet"]
        #print(json)
        columns = json["resultSets"]["headers"][1]['columnNames']
        #print(columns)
        df = pd.DataFrame.from_records(data, columns=columns)

    time.sleep(.25)
    return df


def pull_game_level_team(dates, start_year,end_year):
    frames = []
    shotcolumns = ['FGA_FREQUENCY', 'FGM', 'FGA', 'FG_PCT', 'EFG_PCT', 'FG2A_FREQUENCY', 'FG2M', 'FG2A', 'FG2_PCT', 
                   'FG3A_FREQUENCY', 'FG3M', 'FG3A', 'FG3_PCT']
    
    for year in range(start_year, end_year):
        year_frame = []
        year_dates = [date for date in dates if str(date)[0:4]==str(year)]
        season = str(year - 1) + '-' + str(year)[-2:]
        
        for date in year_dates:
            date_num = int(date)
            date = format_date_to_url(date)
            # Adjust URLs to use team endpoints instead of player endpoints
            url = f'https://stats.nba.com/stats/leaguedashteamstats?College=&Conference=&Country=&DateFrom={date}&DateTo={date}&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
            df = pull_data(url)
            
            url2 = f'https://stats.nba.com/stats/leaguedashteamstats?College=&Conference=&Country=&DateFrom={date}&DateTo={date}&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
            df2 = pull_data(url2)
           

            frames = [df2]
            for frame in frames:
                
                joined_columns = set(frame.columns) - set(df.columns)
                joined_columns = list(joined_columns)
                joined_columns.append('TEAM_ID')
                frame = frame[joined_columns]

                df = df.merge(frame, on='TEAM_ID',how='left').reset_index(drop=True)

            df['year'] = year
            df['date']=date_num
            print(date)

            year_frame.append(df)

        yeardata=pd.concat(year_frame)
        yeardata.to_csv(str(year)+'team.csv',index=False)
        frames.append(yeardata)
        print(f"Year: {year}")

    total = pd.concat(frames)
    return total

def get_dates(start_year,end_year):
    dates=[]
    for year in range(start_year,end_year):
    
        for team in teams.get_teams():
            team_id=team['id']
            path = '../team/'+str(year)+'ps/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['PLAYER_ID','HTM','VTM','GAME_DATE']]
                df.drop_duplicates(inplace=True)
                dates.append(df)
    return pd.concat(dates)
start_year=1997
end_year=2014

dateframe=get_dates(start_year,end_year)
dates=dateframe['GAME_DATE'].unique().tolist()


df= pull_game_level_team(dates,start_year,end_year)


04%2F25%2F1997
04%2F27%2F1997
04%2F29%2F1997
05%2F02%2F1997
05%2F04%2F1997
05%2F06%2F1997
05%2F08%2F1997
05%2F10%2F1997
05%2F11%2F1997
05%2F13%2F1997
04%2F30%2F1997
05%2F20%2F1997
05%2F22%2F1997
05%2F24%2F1997
05%2F26%2F1997
05%2F28%2F1997
06%2F01%2F1997
06%2F04%2F1997
06%2F06%2F1997
06%2F08%2F1997
06%2F11%2F1997
06%2F13%2F1997
04%2F24%2F1997
04%2F26%2F1997
05%2F05%2F1997
05%2F07%2F1997
05%2F09%2F1997
05%2F15%2F1997
05%2F17%2F1997
05%2F19%2F1997
05%2F21%2F1997
05%2F23%2F1997
05%2F25%2F1997
05%2F27%2F1997
05%2F29%2F1997
04%2F28%2F1997
05%2F12%2F1997
05%2F01%2F1997
05%2F14%2F1997
05%2F16%2F1997
05%2F18%2F1997
05%2F03%2F1997
Year: 1997
04%2F23%2F1998
04%2F25%2F1998
04%2F28%2F1998
05%2F01%2F1998
04%2F27%2F1998
04%2F30%2F1998
04%2F24%2F1998
04%2F26%2F1998
04%2F29%2F1998
05%2F03%2F1998
05%2F06%2F1998
05%2F08%2F1998
05%2F10%2F1998
05%2F13%2F1998
05%2F17%2F1998
05%2F19%2F1998
05%2F23%2F1998
05%2F25%2F1998
05%2F27%2F1998
05%2F29%2F1998
05%2F31%2F1998
06%2F03%2F1998
06%2F05%2F1998
06%2F07%2F1998